In [ ]:
import os
import gc
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from datetime import datetime, date
from collections import defaultdict, Counter
from multiprocessing import cpu_count, Pool

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import warnings
warnings.simplefilter('ignore')

In [ ]:
PATH = os.path.join('..', 'data')

KEY = 'card_id'

SEED = 18
# SEED = np.random.randint(9999)

NTHREAD = cpu_count()

NFOLD = 4

In [ ]:
features = os.listdir('../feature')

In [ ]:
train = pd.read_csv(os.path.join(PATH, 'train.csv'))

for f in features:
    print(f'Merge: {f}', end=' ')
    t = pd.read_pickle(os.path.join('..', 'feature', f))
    train = pd.merge(train, t, on=KEY, how='left')
    print('Done!!')

for f in [
    'hist_purchase_date_max', 'hist_purchase_date_min', 
    'N_hist_auth_purchase_date_max', 'N_hist_auth_purchase_date_min', 
    'Y_hist_auth_purchase_date_max', 'Y_hist_auth_purchase_date_min', 
#     'new_purchase_date_max', 'new_purchase_date_min', 
    'N_new_auth_purchase_date_max', 'N_new_auth_purchase_date_min',
    'Y_new_auth_purchase_date_max', 'Y_new_auth_purchase_date_min', 
#     'union_purchase_date_max', 'union_purchase_date_min', 
    'N_union_auth_purchase_date_max', 'N_union_auth_purchase_date_min',
    'Y_union_auth_purchase_date_max', 'Y_union_auth_purchase_date_min']:
    train[f] = train[f].astype(np.int64) * 1e-9

In [ ]:
train.columns

In [ ]:
train.columns.unique()

In [ ]:
cnt = Counter(train.columns)

In [ ]:
for t in cnt.most_common(14):
    print(f"'{t[0]}',")

In [ ]:
drop_cols = [
   'N_authorized_flag_x',
    'Y_authorized_flag_x',
    'N_authorized_flag_y',
    'Y_authorized_flag_y',
    'union_transactions_count_x',
    'union_transactions_count_y',
    'hist_month_lag_mean_x',
    'hist_month_lag_std_x',
    'hist_month_diff_mean_x',
    'hist_month_diff_std_x',
    'hist_month_lag_mean_y',
    'hist_month_lag_std_y',
    'hist_month_diff_mean_y',
    'hist_month_diff_std_y', 
]

In [ ]:
train.drop(drop_cols, axis=1, inplace=True)

In [ ]:
train.columns.unique()

In [ ]:
train = train.loc[:, ~(train.dtypes == object)]

In [ ]:
train.head()

In [ ]:
del train['target']

In [ ]:
gc.collect()

In [ ]:
train = train.fillna(0)

In [ ]:
def drop_duplicated_columns(train):
    drop_cols = set()
    for ei,ci in enumerate(tqdm(train.columns)):
        ti = train[ci].values
        for cj in train.columns[ei+1:]:
            tj = train[cj].values
            if (np.max(ti) != np.min(tj)) or (np.min(ti) != np.min(tj)):
                continue
            if sum(ti != tj) == 0:
                print(ci, cj)
                drop_cols.add(cj)
                
    return drop_cols

In [ ]:
train.dtypes.unique()

In [ ]:
train_int64 = train.loc[:, (train.dtypes == 'int64')]
int64 = drop_duplicated_columns(train_int64)
del train_int64

In [ ]:
train_float64 = train.loc[:, (train.dtypes == 'float64')]
float64 = drop_duplicated_columns(train_float64)
del train_float64

In [ ]:
train_float16 = train.loc[:, (train.dtypes == 'float16')]
float16 = drop_duplicated_columns(train_float16)
del train_float16

In [ ]:
train_float32 = train.loc[:, (train.dtypes == 'float32')]
float32 = drop_duplicated_columns(train_float32)
del train_float32

In [ ]:
train_int8 = train.loc[:, (train.dtypes == 'int8')]
int8 = drop_duplicated_columns(train_int8)
del train_int8

In [ ]:
train_int16 = train.loc[:, (train.dtypes == 'int16')]
int16 = drop_duplicated_columns(train_int16)
del train_int16

In [ ]:
train_uint8 = train.loc[:, (train.dtypes == 'uint8')]
uint8 = drop_duplicated_columns(train_uint8)
del train_uint8

In [ ]:
set(float64)

In [ ]:
set(float16)

In [ ]:
set(float32)

In [ ]:
pd.DataFrame(list(set(float64)), columns={'duplicated_columns'}).head()

In [ ]:
int64

In [ ]:
float16

In [ ]:
float32

In [ ]:
float64

In [ ]:
duplicated_columns = set()

In [ ]:
duplicated_columns = duplicated_columns.union(float16)

In [ ]:
duplicated_columns = duplicated_columns.union(float32)

In [ ]:
duplicated_columns = duplicated_columns.union(float64)

In [ ]:
duplicated_columns = duplicated_columns.union(int8)

In [ ]:
duplicated_columns = duplicated_columns.union(int16)

In [ ]:
duplicated_columns = duplicated_columns.union(int64)

In [ ]:
duplicated_columns = duplicated_columns.union(uint8)

In [ ]:
duplicated_columns = duplicated_columns.union(set(
    ['N_authorized_flag_x',
    'Y_authorized_flag_x',
    'N_authorized_flag_y',
    'Y_authorized_flag_y',
    'union_transactions_count_x',
    'union_transactions_count_y',
    'hist_month_lag_mean_x',
    'hist_month_lag_std_x',
    'hist_month_diff_mean_x',
    'hist_month_diff_std_x',
    'hist_month_lag_mean_y',
    'hist_month_lag_std_y',
    'hist_month_diff_mean_y',
    'hist_month_diff_std_y']))

In [ ]:
len(duplicated_columns)

In [ ]:
duplicated = pd.DataFrame(list(duplicated_columns), columns={'duplicated_columns'})

In [ ]:
duplicated.head()

In [ ]:
duplicated.to_csv('../py/duplicated_columns.py')

In [ ]:
len(train.columns)

In [ ]:
train_M = train.loc[:, (train.dtypes == '<M8[ns]')]

In [ ]:
train_M

In [ ]:
duplicated.duplicated_columns.values